In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Leer datos de entrenamiento

In [2]:
path = 'datacontinuousstudents.csv'
data = pd.read_csv(path, sep = ',', na_filter = False)
data = data.set_index('id')

X = data.drop(columns = ['y']).copy()
y_train = data[['y']].copy()

In [3]:
display(X.head())
display(y_train.head())

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32
id,,,,,,,,,,,,,,,,,,,,,
1,49.856,118.8,1,0,0,0,0,16.32096,0,18.762237,...,0.9709,0.0003,6,6,24.7138,0,0,7,0.056546,0.112860
2,71.836,34.8,0,0,0,0,1,15.00535,44,119.946790,...,0.4200,0.0000,8,4,38.0658,0,0,7,0.655260,0.886908
3,66.725,41.8,0,1,0,0,0,17.07023,27,10.879595,...,0.9650,0.0000,6,6,48.3807,0,1,4,0.067855,0.133944
4,41.696,132.0,0,0,1,0,0,16.31460,0,9.762927,...,0.0091,0.0001,6,4,59.8410,0,0,7,0.133230,0.260102
5,71.351,24.2,1,0,0,0,0,11.10790,31,151.581820,...,0.0051,0.0000,5,7,175.9250,0,0,3,0.318584,0.627729


,y
id,
1,-0.852667
2,0.685760
3,5.951199
4,-2.030599
5,-6.143410


# Leer variables

In [4]:
modelToProof = [['x12','x23','x25','x32']]
modelToProof.append(['x6','x7','x12','x23','x24','x25','x32'])
modelToProof.append(['x6','x13','x20','x23','x25','x27','x31'])
modelToProof.append(['x1','x23','x25','x32'])
modelToProof.append(['x1','x23','x25','x31'])
modelToProof.append(['x12','x23','x24','x25','x32'])
modelToProof.append(['x6','x7','x12','x23','x24','x25','x32'])
modelToProof.append(['x23','x25','x31'])
modelToProof.append(['x13','x23','x25'])
modelToProof.append(['x23','x25'])

In [5]:
def calculated_metrics(variablesSelected,X_trained_scaled,y_train,X_test_scaled,y_test):
    X_trained_scaled_aux = X_trained_scaled[variablesSelected].copy()
    X_test_scaled_aux = X_test_scaled[variablesSelected].copy()
    ## Regresion para cada cantidad de variables
    regr = LinearRegression()

    # Train the model using the training sets
    regr.fit(X_trained_scaled_aux,  y_train['y'].values)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test_scaled_aux)

    y_pred = pd.DataFrame(y_pred, index=y_test.index, columns=["y_pred"])

    total = y_pred.join(y_test)
    total['y_menor_-1'] = (total['y']<-1).astype(int)
    total['y_pred_menor_-1'] = (total['y_pred']<-1).astype(int)
    total['good_class'] = (total['y_menor_-1']==total['y_pred_menor_-1']).astype(int)
    mse = mean_squared_error(y_test, y_pred)
    accuray =total['good_class'].sum()/len(total)
    r2 = r2_score(y_test, y_pred)
    return mse,accuray,r2

In [12]:
resultados =  pd.DataFrame()
for variablesSelected in modelToProof:
    print(variablesSelected)
    mse = []
    accuray = []
    r2 = []
    if len(variablesSelected)>0:
        for j in range(1,1000):
            X_train, X_test_aux, y_train_aux, y_test_aux = train_test_split(X, y_train, test_size=0.2)
            standarscaler = StandardScaler()
            X_trained_scaled_aux = standarscaler.fit_transform(X_train)
            X_trained_scaled_aux = pd.DataFrame(X_trained_scaled_aux, index=X_train.index, columns=X_train.columns.values)

            X_test_scaled_aux = standarscaler.transform(X_test_aux)

            X_test_scaled_aux = pd.DataFrame(X_test_scaled_aux, index=X_test_aux.index, columns=X_test_aux.columns.values)


#             variablesSelected = list(variablesTorun[variablesTorun[i]==1]['VariableName'])
            mse_aux,accuray_aux,r2_aux= calculated_metrics(variablesSelected,X_trained_scaled_aux,y_train_aux,X_test_scaled_aux,y_test_aux)
            mse.append(mse_aux)
            accuray.append(accuray_aux)
            r2.append(r2_aux)

        auxRes= pd.DataFrame({'Modelo': [variablesSelected], 'Median: Mean squared error': [np.median(np.asarray(mse))],'Mean: Mean squared error': [np.mean(np.asarray(mse))], 'Median: R2': [np.median(np.asarray(r2))],'Median: Accuray': [np.median(np.asarray(accuray))],'Mean: Accuray': [np.mean(np.asarray(accuray))]})
        resultados = pd.concat([resultados,auxRes])
        resultados.to_csv('ResultadosCV_scores_subset.csv')
    else:
        print(f"El conjunto de variables {variablesSelected} no tiene variables")
#     break
# resultado_scores_ranking_cv.reset_index(inplace = True,drop = True)

['x12', 'x23', 'x25', 'x32']
['x6', 'x7', 'x12', 'x23', 'x24', 'x25', 'x32']
['x6', 'x13', 'x20', 'x23', 'x25', 'x27', 'x31']
['x1', 'x23', 'x25', 'x32']
['x1', 'x23', 'x25', 'x31']
['x12', 'x23', 'x24', 'x25', 'x32']
['x6', 'x7', 'x12', 'x23', 'x24', 'x25', 'x32']
['x23', 'x25', 'x31']
['x13', 'x23', 'x25']
['x23', 'x25']


In [13]:
resultados.sort_values('Median: Mean squared error',ascending = True)

,Modelo,Median: Mean squared error,Mean: Mean squared error,Median: R2,Median: Accuray,Mean: Accuray
0,"[x6, x13, x20, x23, x25, x27, x31]",3.612060,3.675031,0.194646,0.733333,0.733166
0,"[x1, x23, x25, x31]",3.812298,3.896753,0.141562,0.766667,0.754321
0,"[x23, x25, x31]",3.854182,3.924263,0.150421,0.766667,0.751051
0,"[x12, x23, x24, x25, x32]",3.862812,3.903222,0.140511,0.766667,0.775709
0,"[x6, x7, x12, x23, x24, x25, x32]",3.882322,3.918198,0.151292,0.766667,0.768869
0,"[x1, x23, x25, x32]",3.887315,3.921212,0.144333,0.766667,0.754054
0,"[x12, x23, x25, x32]",3.925078,3.972765,0.141036,0.766667,0.754288
0,"[x6, x7, x12, x23, x24, x25, x32]",3.933950,3.968389,0.126079,0.766667,0.760694
0,"[x13, x23, x25]",4.141742,4.168143,0.079084,0.766667,0.767401
0,"[x23, x25]",4.143075,4.177026,0.081930,0.766667,0.774274
